# Module initialization

In [1]:
%cd ../covid_households
import recipes
import interventions
import traits

/Users/thayer/covid_households/covid_households


# Vaccine trial

This cell configures the parameters that are shared between the vaccinated arm and the control arm.

The traits of susceptibility and infectivity refer to population variation in these traits of individuals (i.e. it's unrelated to the vaccine). You can set these to a `ConstantTrait` for no variation or to a `LognormalTrait` to achieve variation in the population.

In [2]:
susceptibility = traits.ConstantTrait()
#susceptibility = traits.LognormalTrait.from_natural_mean_variance(mean=1.0, variance=0.5)
infectivity = traits.ConstantTrait()
#infectivity = traits.LognormalTrait.from_natural_mean_variance(mean=1.0, variance=2.0)

# {size: # of households of that size}
sizes = {2:100, 3:100}
trials = 10
household_beta = 0.05

This cell configures the vaccine.

The `shape` refers to how the vaccine is applied to the population. `InterveneOnFirst` means the "first" individual in each household is vaccinated (since household order is totally random, this effectively vaccinates a random individual in each household, but be careful not to break the symmetry of individuals by introducing something *else* that cares about particular individuals).

The `vaccine` determines the (relative per-contact) susceptibility and infectivity of vaccinated individuals. For example, `sus_factor=0.2` says that a vaccinated person is only $20\%$ as likely as an unvaccinated person to be infected per contact.

In [3]:
shape = interventions.InterveneOnFirst()
vaccine = interventions.ConstantFactorIntervention(shape, sus_factor=0.2, inf_factor=0.3)

These cells simulate forwards in time. They are purely configured above, you shouldn't need to touch them except for advanced uses.

In [4]:
vax_model = recipes.Model(intervention=vaccine)
vax_df = vax_model.run_trials(household_beta=household_beta, sizes=sizes, trials=trials, sus=susceptibility, inf=infectivity, as_counts=False)

vax_df.groupby('trial').sum()

,size,infections,intervention and infection,total interventions
trial,,,,
0,500,246,50,200.0
1,500,236,39,200.0
2,500,245,43,200.0
3,500,239,34,200.0
4,500,243,35,200.0
5,500,234,33,200.0
6,500,249,41,200.0
7,500,238,42,200.0
8,500,235,40,200.0


In [5]:
# 1.0 and 1.0 because the placebo has no effect
control_model = recipes.Model(intervention=interventions.ConstantFactorIntervention(shape, 1.0, 1.0))
control_df = control_model.run_trials(household_beta=household_beta, sizes=sizes, trials=trials, sus=susceptibility, inf=infectivity, as_counts=False)

control_df.groupby('trial').sum()

,size,infections,intervention and infection,total interventions
trial,,,,
0,500,310,119,200.0
1,500,282,113,200.0
2,500,323,131,200.0
3,500,286,114,200.0
4,500,302,115,200.0
5,500,309,123,200.0
6,500,303,120,200.0
7,500,302,124,200.0
8,500,293,119,200.0


# Quantifying vaccine effects

Defining and calculating $\text{VE}_{\text{S}}$:

$$ \text{VE}_{\text{S}} = \left.1 - \frac{AR_v}{AR_p} = 1 - {n_v^+\ /\ (n_v^+ + n_v^-)} \middle/ \right. {n_p^+\ /\ (n_p^+ + n_{p}^-)}{}$$

Defining and calculating $\text{VE}_{\text{contacts}}$:

$$ \text{VE}_{\text{contact}} = \left.1 - \frac{AR_u}{AR_{np}} = 1 - {\ n_u^+/\ (n_u^+ + n_u^-)} \middle/ \right. {n_{np}^+\ /\ (n_{np}^+ + n_{np}^-)}$$

where $\pm$ refers to infected and uninfected individuals, $n_v$ refers to vaccinated individuals (who received the real vaccine), $n_u$ refers to unvaccinated individuals (in households where the vaccine was administered), $n_p$ refers to placebo-receiving individuals, and $n_{np}$ refers to individuals who received no placebo (in households where the placebo was administered).

The term $AR$ (attack rate) is used as defined in the literature and should not be understood as a rate, but simply as the observed frequency of infections among a group such that $AR_v$, for example, is defined as the fraction of vaccinated individuals who were in fact infected.

Defining and calculating $\text{VE}_{\text{total}}$:

$$ \text{VE}_{\text{total}} = \left.1 - \frac{AR_{HV}}{AR_{HP}} = 1 - {n_{HV}^+\ /\ (n_{HV}^+ + n_{HV}^-)} \middle/ \right. {n_{HP}^+\ /\ (n_{HP}^+ + n_{HP}^-)}{}$$

Heree the notation has changed slightly to allow $n_{HV}$ to refer to the total number of individuals in households that received a vaccine, ie $n_{HV} = n_{v} + n_{u}$.

In [10]:
import pandas as pd
import scipy

def ves(vax_df, control_df):
    """Vaccination effect on susceptibility using the placebo RR as baseline (equation 1/2 in Betz)"""
    print("Calculating VEs ...\n")

    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["intervention and infection"].sum() / vg["total interventions"].sum()

    cg = control_df.groupby(["trial"])
    cgs = cg.sum()
    f_c = cg["intervention and infection"].sum() / cg["total interventions"].sum()     

    # fisher exact test record actual number of events: columns either vaccinated or in household with vaccination vs other and rows = individual was infected vs not

    # fisher exact test : comparing primary participants in households

    ##             placebo | vaccinated
    ##  uninfected
    ##  -----
    ##  infected

    fisher_df = pd.concat([cgs["total interventions"] - cgs["intervention and infection"], vgs["total interventions"] - vgs["intervention and infection"], cgs["intervention and infection"], vgs["intervention and infection"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    ve = 1. - f_v / f_c
    ve.name = "VE"

    return pd.concat([ve, p], axis=1)

def vecontact(vax_df, control_df):
    print("Calculating VEcontact ...\n")
    vax_df = vax_df.copy()
    vax_df["total unvaccinated"] = vax_df["size"] - vax_df["total interventions"]
    vax_df["unvaccinated and infected"] = vax_df["infections"] - vax_df["intervention and infection"]
    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["unvaccinated and infected"].sum() / vg["total unvaccinated"].sum()

    control_df = control_df.copy()
    cg = control_df.groupby(["trial"])
    control_df["total unvaccinated"] = control_df["size"] - control_df["total interventions"]
    control_df["unvaccinated and infected"] = control_df["infections"] - control_df["intervention and infection"]
    cgs = cg.sum()
    f_c = cg["unvaccinated and infected"].sum() / cg["total unvaccinated"].sum()

    import pdb; pdb.set_trace()
    
    ve = 1. - f_v / f_c
    #ve.name = "VEcontact"
    ve.name = "VE"

    # fisher exact test : comparing households by type but only unvaccinated
    ##             control hh secondary (no placebo) | vaccinated hh secondary (no vax)
    ##  uninfected
    ##  -----
    ##  infected


    fisher_df = pd.concat([cgs["total unvaccinated"]-cgs["unvaccinated and infected"], vgs["total unvaccinated"]-vgs["unvaccinated and infected"], cgs["unvaccinated and infected"], vgs["unvaccinated and infected"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    return pd.concat([ve, p], axis=1)

def vetotal(vax_df, control_df):
    print("Calculating VEtotal ...\n")
    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["infections"].sum() / vg["size"].sum()

    cg = control_df.groupby(["trial"])
    cgs = cg.sum()
    f_c = cg["infections"].sum() / cg["size"].sum()

    ve = 1. - (f_v)/(f_c)
    #ve.name = "VEtotal"
    ve.name = "VE"

    # fisher exact test : comparing households by type

            ##             control hh | vaccinated hh
    ##  uninfected
    ##  -----
    ##  infected


    fisher_df = pd.concat([cgs["size"]-cgs["infections"], vgs["size"]-vgs["infections"], cgs["infections"], vgs["infections"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    return pd.concat([ve, p], axis=1)

In [11]:
ves(vax_df, control_df)

Calculating VEs ...



,VE,fisher p value
trial,,
0,0.579832,3.396200e-12
1,0.654867,2.220281e-14
2,0.671756,4.026322e-19
3,0.701754,7.300144e-17
4,0.695652,9.172999e-17
5,0.731707,1.008842e-20
6,0.658333,6.363306e-16
7,0.661290,6.377004e-17
8,0.663866,5.413028e-16


In [ ]:
vecontact(vax_df, control_df)

Calculating VEcontact ...

> /var/folders/9w/8b0b4bqn1vv4gl40h13fzdr40000gn/T/ipykernel_8848/3030056802.py(53)vecontact()
     51     import pdb; pdb.set_trace()
     52 
---> 53     ve = 1. - f_v / f_c
     54     #ve.name = "VEcontact"
     55     ve.name = "VE"

ipdb> vax_df
      size  infections  intervention and infection  total interventions  \
0        2           1                           0                  1.0   
1        2           1                           1                  1.0   
2        2           1                           1                  1.0   
3        2           1                           1                  1.0   
4        2           1                           0                  1.0   
...    ...         ...                         ...                  ...   
1995     3           1                           0                  1.0   
1996     3           1                           0                  1.0   
1997     3           1                        

In [9]:
vetotal(vax_df, control_df)

Calculating VEtotal ...



,VE,fisher p value
trial,,
0,0.206452,5.943969e-05
1,0.163121,4.377484e-03
2,0.241486,8.397977e-07
3,0.164336,3.559413e-03
4,0.195364,2.267430e-04
5,0.242718,2.524537e-06
6,0.178218,7.426251e-04
7,0.211921,6.269552e-05
8,0.197952,3.011070e-04
